# Comparing GO and KM results

In [1]:
#imports
import os
import csv

In [2]:
#dictionary of csv paths
def get_paths(GO_dir, batch):
    csv_paths = {}
    batch_dir = os.path.join(GO_dir, batch)
    for f in os.listdir(batch_dir):
        #get all directories
        cluster_dir = os.path.join(batch_dir, f)
        if os.path.isdir(cluster_dir):
            cluster_dir_name = f.split('_prerank')[0]
            for g in os.listdir(cluster_dir):
                if g[-4:] == '.csv': #found csv
                    csv_paths[cluster_dir_name] = os.path.join(cluster_dir, g)
    return csv_paths

In [3]:
def filter_metagenes(csv_paths, pval_cutoff, fdr_cutoff, filename):
    diff_exp = {}
    dex = []
    
    csv_columns = None
    for comp, csv_f in csv_paths.items():
        diff_exp[comp] = []
        with open(csv_f) as f:
            input_file = csv.DictReader(f)
            for gene_set in input_file:
                if csv_columns is None:
                    csv_columns = ["pair"] + list(gene_set.keys()) 
                try:
                    if float(gene_set['pval']) < pval_cutoff and float(gene_set['es'])*float(gene_set['fdr']) < fdr_cutoff:
                        diff_exp[comp].append(gene_set) 
                        
                        gene_set['pair'] = comp
                        dex.append(gene_set)
                #ignore results that are missing a p-val 
                except:
                    pass
        diff_exp[comp] = sorted(diff_exp[comp], key = lambda x: float(x['es']), reverse = True) #sort by enrichment
    
    #write filtered to csv
    with open(os.path.join(GO_dir, filename), 'w', newline="") as f:  
        writer = csv.DictWriter(f, fieldnames=csv_columns)
        writer.writeheader()
        for data in dex:
            writer.writerow(data)
    
    return diff_exp
    

In [4]:
# read in KM data, filter on KM
def survival_data(survival_analysis, HZ_cutoff, FC_cutoff):
    survival_data = {}
    with open(survival_analysis) as f:
        input_file = csv.DictReader(f)
        for KM in input_file:
            if float(KM['HZ_KM']) > HZ_cutoff and KM['signature'][-1] == str(FC_cutoff): #only consider fold change threshold 1 for now
                survival_data[KM['signature']] = (float(KM['pval_KM']), float(KM['HZ_KM']), int(KM['n_low']), int(KM['n_high']))
    return survival_data

In [5]:
#read in metagenes, enforce fc 2 cutoff
def read_metagenes(metagene_dir, FC_cutoff ):
    metagene_dict = {}
    
    for subdir in ["/UP", "/DOWN"]:
        for file in os.listdir(metagene_dir + subdir):
            metagene_dict[file.replace(".csv", "")] = []
            with open(os.path.join(metagene_dir + subdir, file)) as f:
                input_file = csv.DictReader(f)
                for gene in input_file:
                    if float(gene['logfoldchange']) > FC_cutoff:
                        metagene_dict[file.replace(".csv", "")].append(gene['genes'])

    return metagene_dict

In [6]:
# for each survival analysis that looks interesting, look at overlap in GO enriched vs our metagenes
def calc_overlap(filename, survival_data, diff_exp, metagene_dict):
    results = []
    for KM in survival_data:
        key = '_'.join(KM.split('_')[1:-2])
        if key not in diff_exp:
            continue
        for gene_group in diff_exp[key]:
            gene_list = set(gene_group['ledge_genes'].split(';'))
            metagene = metagene_dict[key]
            overlap = [mg for mg in metagene if mg in gene_list]
            result_dict = {'key':key, 'term':gene_group['Term'], 'es':gene_group['es'], 'overlap':";".join(overlap), 
                           'ledge':gene_group['ledge_genes'], 'metagene':";".join([mg for mg in metagene]), 
                            'perc': len(overlap) / float(len(gene_list))}
            results.append(result_dict)
            
    csv_columns = ['key', 'term', 'es', 'overlap', 'ledge', 'metagene', 'perc']
    
    #write filtered to csv
    with open(os.path.join(GO_dir, filename), 'w', newline="") as f:  
        writer = csv.DictWriter(f, fieldnames=csv_columns)
        writer.writeheader()
        for result_dict in results:
            writer.writerow(result_dict)

In [7]:
def run_all(GO_dir, batch, metagene_dir, survival_analysis, pval_cutoff, fdr_cutoff, hz_cutoff, fc_cutoff):
    csv_paths = get_paths(GO_dir, batch)
    diff_exp = filter_metagenes(csv_paths, pval_cutoff, fdr_cutoff, f'{batch}_GO_filtered.csv')
    sa = survival_data(survival_analysis, hz_cutoff, fc_cutoff)
    metagene_dict = read_metagenes(metagene_dir, fc_cutoff)
    calc_overlap(f"{batch}_overlap_FC-{fc_cutoff}_HZ-{hz_cutoff}.csv", sa, diff_exp, metagene_dict)

In [8]:
#filepaths
GO_dir = "/Users/student/Documents/Wi20-SysBio/BP205B_2020/GO-KM_overlap"
filepaths = [["MDA", "/Users/student/Documents/Wi20-SysBio/metagenes/mda", "/Users/student/Documents/Wi20-SysBio/BP205B_2020/GO-KM_overlap/MDA_filtered.csv"],
             ["HCC_MK", "/Users/student/Documents/Wi20-SysBio/metagenes/mk",  "/Users/student/Documents/Wi20-SysBio/BP205B_2020/GO-KM_overlap/HCC_filtered.csv"]]

pval_cutoff, fdr_cutoff = 0.05, 0.05 

fc_cutoffs = [0, 1, 2]
hz_cutoffs = [0.5, 0.7, 1]
for batch_info in filepaths:
    for fc_cutoff in fc_cutoffs:
        for hz_cutoff in hz_cutoffs:
            run_all(GO_dir, *batch_info, pval_cutoff, fdr_cutoff, hz_cutoff, fc_cutoff)